In [7]:
from enum import StrEnum, Enum, auto
from IPython.display import display
import ipywidgets as widgets

# ──────────────── Model ────────────────
class Cell(StrEnum):
    EMPTY = ' '
    NOUGHT = 'O'
    CROSS = 'X'

class Status(Enum):
    UNDERWAY = auto()
    DRAW = auto()
    NOUGHT_WIN = auto()
    CROSS_WIN = auto()

class Board:
    """In 3 * 3 layout"""
    def __init__(self):
        self.board: list[list[Cell]] = [[Cell.EMPTY for _ in range(3)] for _ in range(3)]
        self.status: Status = Status.UNDERWAY

    def __getitem__(self, pos) -> Cell:
        row, col = pos
        return self.board[row][col]
    
    def __setitem__(self, pos, val: Cell):
        row, col = pos
        self.board[row][col] = val

    def judge(self):
        """Check if one side wins, loses, or both draw."""
        for row in self.board:
            if row[0] == row[1] == row[2] != Cell.EMPTY:
                self.status = Status.NOUGHT_WIN if row[0] == Cell.NOUGHT else Status.CROSS_WIN
                return
        for col in zip(*self.board):
            if col[0] == col[1] == col[2] != Cell.EMPTY:
                self.status = Status.NOUGHT_WIN if col[0] == Cell.NOUGHT else Status.CROSS_WIN
                return
        if self.board[0][0] == self.board[1][1] == self.board[2][2] != Cell.EMPTY:
            self.status = Status.NOUGHT_WIN if self.board[1][1] == Cell.NOUGHT else Status.CROSS_WIN
            return
        if self.board[2][0] == self.board[1][1] == self.board[0][2] != Cell.EMPTY:
            self.status = Status.NOUGHT_WIN if self.board[1][1] == Cell.NOUGHT else Status.CROSS_WIN
            return
        if not any(Cell.EMPTY in row for row in self.board):
            self.status = Status.DRAW
            return

# ────────────── Controller ───────────────
class TicTacToeGame:
    def __init__(self):
        self.board = Board()
        self.current_player = Cell.CROSS  # 'X' first

    def make_move(self, row: int, col: int) -> bool:
        """Try put 'O' or 'X' into a cell"""
        if self.board.status != Status.UNDERWAY:
            return False
        if self.board[row, col] != Cell.EMPTY:
            return False
        self.board[row, col] = self.current_player
        self.board.judge()
        if self.board.status == Status.UNDERWAY:
            self.current_player = Cell.NOUGHT if self.current_player == Cell.CROSS else Cell.CROSS
        return True

    def reset(self):
        self.board = Board()
        self.current_player = Cell.CROSS

    @property
    def is_over(self) -> bool:
        return self.board.status != Status.UNDERWAY

    @property
    def status_message(self) -> str:
        if self.board.status == Status.UNDERWAY:
            return f"👉 轮到玩家 {self.current_player}'s turn"
        elif self.board.status == Status.DRAW:
            return "🚧 平局Draw！"
        else:
            winner = "O" if self.board.status == Status.NOUGHT_WIN else "X"
            return f"🎉 玩家 {winner} 获胜Win！"


In [8]:
# ──────────────── View ────────────────
class TicTacToeUI:
    def __init__(self, game: TicTacToeGame):
        self.game = game
        self.buttons = [[self._create_a_button(r, c) for c in range(3)] for r in range(3)]
        
        self.board_grid = widgets.GridBox(
            [self.buttons[r][c] for r in range(3) for c in range(3)],
            layout=widgets.Layout(grid_template_columns="repeat(3, 60px)")
        )
        
        self.info_label = widgets.Label(value=self.game.status_message)
        self.reset_button = widgets.Button(description="🔄 重新开始Restart", button_style='info')
        self.reset_button.on_click(self._on_reset)

        self.container = widgets.VBox([self.board_grid, self.info_label, self.reset_button])
        self._update_ui()

    def _create_a_button(self, r: int, c: int) -> widgets.Button:
        btn = widgets.Button(
            description=' ',
            layout=widgets.Layout(width='57px', height='60px', font_size='24px')
        )
        btn.on_click(lambda _: self._on_cell_click(r, c))
        return btn

    def _on_cell_click(self, r: int, c: int):
        if self.game.make_move(r, c):
            self._update_ui()

    def _on_reset(self, _):
        self.game.reset()
        self._update_ui()

    def _update_ui(self):
        # Refresh
        for r in range(3):
            for c in range(3):
                cell = self.game.board[r, c]
                btn = self.buttons[r][c]
                btn.description = str(cell)
                btn.disabled = (self.game.is_over or cell != Cell.EMPTY)

        self.info_label.value = self.game.status_message

    def display(self):
        display(self.container)


game = TicTacToeGame()
ui = TicTacToeUI(game)
ui.display()